# Descarga de datos de Fallecimientos desde Sanidad

## Objetivo

Vamos a hacer una lectura de los fallecimientos comunicados por sanidad en cada uno de los pdf que publica

V3: versión definitiva con

## Código

Tenemos un montón de librerias con las que vamos a trabajar, no he tenido ningún problema en instalar aquellas no disponibles en el entorno Anaconda con el que trabajo por medio de `pip install libreria` realizado desde un terminal abierto desde el entorno `env`.

In [1]:
import os.path as pth
import datetime as dt
import time
from glob import glob
import re
import pandas as pd
import numpy as np

import requests
from shutil import copyfile

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, WeekdayLocator, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import seaborn as sns
import matplotlib.colors as colors

import numpy as np
from datetime import datetime
import seaborn as sns
%matplotlib inline

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

Preparamos las expresiones regulares que nos ayudarán en la interpretación de la información que sacamos de los pdf.

Definimos variables que nos ayuden en la gestión de los nombres de los ficheros.

In [2]:
datadir='datos_sanidad/'
URL_reg='https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_{:02d}_COVID-19.pdf'

# Actualización día ZERO con el histórico

### Función de para descargar un fichero pdf

In [3]:
def descarga(url,num):
    print('Descargando:', url)
    fn=datadir+str(num)+'.pdf'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    with requests.Session() as s:
        r = s.get(url, headers=headers)
    if r.status_code == requests.codes.ok:
        with open(fn, 'wb') as fp:
                fp.write(r.content)
    else:
        print ('Error con el ',num)
    return True

### Descarga de un rango o de un solo pdf

In [4]:
for i in range(77,263):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_77_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_78_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_79_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_80_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_81_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_82_COVID-19.pdf
Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_83_COVID-19.pdf


KeyboardInterrupt: 

### Descarga de un solo fichero

In [5]:
descarga(URL_reg.format(267),267)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_267_COVID-19.pdf
Error con el  267


True

### Extraemos el texto de la pagina concreta del pdf

In [6]:
# Extract PDF text using PDFMiner. Adapted from
# http://stackoverflow.com/questions/5725278/python-help-using-pdfminer-as-a-library

def pdf_to_text(pdfname, pagenum=None):

    # PDFMiner boilerplate
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    laparams = LAParams()
    device = None
    try:
        device = TextConverter(rsrcmgr, sio, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # Extract text
        with open(pdfname, 'rb') as fp:
            for i, page in enumerate(PDFPage.get_pages(fp)):
                if pagenum is None or pagenum == i:
                    interpreter.process_page(page)

        # Get text from StringIO
        text = sio.getvalue()
    finally:
        # Cleanup
        sio.close()
        if device is not None:
            device.close()

    return text

### Creamos el pandas datos, al que vamos incorporando los datos leidos

In [7]:
datos=pd.DataFrame()

### Descarga del documento 235 a la actualidad

In [8]:
pattern=re.compile(r'(\n{1,2}\d*[,.]?\d+¥? ){19}')
pattern=re.compile(r'(\n\nTotal) ((\n{1,2}\d*[,.]?\d+¥? ){19})')
for i in range(235,266):
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    text = pdf_to_text(fn, pagenum=4)  #con que pagina queremos trabajar?
    cadena=re.search(pattern,text).group(2)         
    print ('Documento {}-->'.format(i),cadena.replace('¥','').replace('\n','').split())
    datos.loc[:,str(i)]=cadena.replace('¥','').replace('\n','').split()

Documento 235--> ['2.270', '1566', '391', '352', '267', '248', '3.338', '3.446', '5.960', '13', '1.719', '676', '859', '10.211', '11', '283', '668', '2.032', '442']
Documento 236--> ['2.314', '1598', '409', '356', '270', '248', '3.349', '3.502', '5.961', '16', '1.727', '687', '882', '10.247', '11', '288', '675', '2.038', '453']
Documento 237--> ['2.370', '1613', '414', '357', '273', '250', '3.367', '3.528', '5.968', '16', '1.737', '697', '884', '10.327', '11', '299', '687', '2.046', '454']
Documento 238--> ['2.402', '1620', '419', '357', '277', '251', '3.383', '3.549', '5.972', '16', '1.751', '701', '895', '10.350', '12', '300', '694', '2.061', '456']
Documento 239--> ['2.432', '1627', '427', '357', '278', '252', '3.405', '3.567', '5.977', '18', '1.762', '703', '895', '10.403', '12', '306', '699', '2.061', '458']
Documento 240--> ['2.470', '1654', '437', '362', '282', '252', '3.425', '3.599', '5.991', '19', '1.773', '714', '904', '10.419', '12', '317', '703', '2.082', '463']
Documento 

### Aquí tenemos una serie de días que han dado error en el escrapeo y no pude solucionar, luego introducimos los valores manualmente

In [9]:
dia_234=[2183,1559,383,344,266,248,3336,3421,5958,13,1709,670,850,10155,11,279,666,2029,441] 

In [10]:
dia_137=[1404,826,314,209,151,202,2945,1928,5587,4,1332,508,609,8691,2,148,490,1424,362]

In [11]:
datos.loc[:,'234']=dia_234

In [12]:
datos.loc[:,'137']=dia_137

### Ahora metemos la serie antígua..del 77 al 234

In [13]:
texto1="Total"

texto2="\n\n"

for i in range(77,234): #original 100 a 234
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    text = pdf_to_text(fn, pagenum=1)
    #with open(fn1, 'w', encoding='utf-8') as fp:
    #with open(fn1, 'w') as fp:    
    #        fp.write(page1)
 

    #with open(fn1) as fp:
    #    text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
    
    if ((((i >= 122) & (i<=139))) & (i!=137)):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[113])
        #print (i,'ojo',' ->',lista[113:113+19])
        datos.loc[:,str(i)]=lista[113:113+19]
            
    elif (i==151):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[127])
        datos.loc[:,str(i)]=lista[127:127+19]
    elif (i==154):
        lista=text.partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[123])
        datos.loc[:,str(i)]=lista[123:123+19]
        
    elif (i!=137):
        lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
        print (i,' -> ',lista[1])
        datos.loc[:,str(i)]=lista[1:1+19]

        #print (i,' -> ',lista[0],'-->',lista[1:20])

77  ->  912
78  ->  940
79  ->  967
80  ->  993
81  ->  1.013
82  ->  1.017
83  ->  1.050
84  ->  1.079
85  ->  1.107
86  ->  1.131
87  ->  1.145
88  ->  1.157
89  ->  1.168
90  ->  1.188
91  ->  1.207
92  ->  1.238
93  ->  1.253
94  ->  1.256
95  ->  1.263
96  ->  1.267
97  ->  1.281
98  ->  1.294
99  ->  1.301
100  ->  1.317
101  ->  1.320
102  ->  1.322
103  ->  1.326
104  ->  1.332
105  ->  1.336
106  ->  1.344
107  ->  1.355
108  ->  1.358
109  ->  1.358
110  ->  1.358
111  ->  1.371
112  ->  1.375
113  ->  1.377
114  ->  1.389
115  ->  1.391
116  ->  1.334
117  ->  1.404
118  ->  1.404
119  ->  1.404
120  ->  1.404
121  ->  1.404
122  ->  1.404
123  ->  1.404
124  ->  1.404
125  ->  1.404
126  ->  1.404
127  ->  1.404
128  ->  1.404
129  ->  1.404
130  ->  1.404
131  ->  1.404
132  ->  1.404
133  ->  1.404
134  ->  1.404
135  ->  1.404
136  ->  1.404
138  ->  1.404
139  ->  1.404
140  ->  1.404
141  ->  1.426
142  ->  1.426
143  ->  1.426
144  ->  1.426
145  ->  1.426
146  ->  1.

### Filtramos las cilumnas a las fechas correctas: desde 77 hasta hoy, y formateamos el index con la denominación correcta de la CA

In [71]:
datos

Fecha,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-11-20,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-30,2020-12-01,2020-12-02,2020-12-03
Andalucia,912,940,967,993,1013,1017,1050,1079,1107,1131,...,3527,3633,3725,3790,3842,3903,4018,4099,4184,4251
Aragon,543,562,601,619,637,643,656,681,695,709,...,2148,2189,2208,2227,2251,2279,2314,2335,2344,2354
Asturias,168,174,187,196,200,202,211,223,231,239,...,800,857,883,904,929,954,1004,1021,1036,1054
Baleares,131,134,148,155,157,161,164,168,172,174,...,398,398,399,402,404,404,409,411,413,423
Canarias,107,111,115,119,119,120,121,121,128,130,...,322,326,333,334,336,339,344,344,345,346
Cantabria,137,144,149,153,158,165,167,173,178,182,...,293,297,297,298,300,304,312,316,320,321
Castilla La Mancha,1796,1852,1913,1963,2021,2075,2140,2188,2255,2292,...,3659,3687,3693,3722,3738,3761,3775,3786,3810,3819
Castilla y Leon,1372,1401,1429,1458,1493,1521,1554,1582,1612,1639,...,4293,4343,4394,4434,4463,4487,4545,4576,4599,4609
Cataluña,3855,3752,3879,3933,4009,4152,4247,4343,4393,4498,...,7495,7596,7692,7793,7850,7911,7911,8003,8067,8102
Ceuta,4,4,4,4,4,4,4,4,4,4,...,46,49,51,52,52,54,54,54,55,55


In [14]:
datos=datos[[str(i) for i in range(77,266)]]

In [15]:
datos.index=['Andalucia','Aragon','Asturias','Baleares','Canarias','Cantabria','Castilla La Mancha',
            'Castilla y Leon','Cataluña','Ceuta','C.Valenciana','Extremadura','Galicia','Madrid','Melilla','Murcia',
             'Navarra','Pais Vasco','La Rioja']

In [16]:
datos=datos.applymap(lambda x: int(str(x).replace(".","")))

In [21]:
claves=pd.read_excel(datadir+'clave_numero_fecha.xlsx')

In [22]:
claves=claves.loc[claves.index[:]]

In [23]:
datos.columns=claves['Fecha']

In [24]:
datos.to_csv('datos_sanidad_matriz.csv')

In [25]:
datos

Fecha,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04
Andalucia,912,940,967,993,1013,1017,1050,1079,1107,1131,...,3633,3725,3790,3842,3903,4018,4099,4184,4251,4296
Aragon,543,562,601,619,637,643,656,681,695,709,...,2189,2208,2227,2251,2279,2314,2335,2344,2354,2360
Asturias,168,174,187,196,200,202,211,223,231,239,...,857,883,904,929,954,1004,1021,1036,1054,1073
Baleares,131,134,148,155,157,161,164,168,172,174,...,398,399,402,404,404,409,411,413,423,425
Canarias,107,111,115,119,119,120,121,121,128,130,...,326,333,334,336,339,344,344,345,346,346
Cantabria,137,144,149,153,158,165,167,173,178,182,...,297,297,298,300,304,312,316,320,321,324
Castilla La Mancha,1796,1852,1913,1963,2021,2075,2140,2188,2255,2292,...,3687,3693,3722,3738,3761,3775,3786,3810,3819,3825
Castilla y Leon,1372,1401,1429,1458,1493,1521,1554,1582,1612,1639,...,4343,4394,4434,4463,4487,4545,4576,4599,4609,4647
Cataluña,3855,3752,3879,3933,4009,4152,4247,4343,4393,4498,...,7596,7692,7793,7850,7911,7911,8003,8067,8102,8126
Ceuta,4,4,4,4,4,4,4,4,4,4,...,49,51,52,52,54,54,54,55,55,55


In [26]:
datos_tabla=datos.unstack().reset_index()[['level_1','Fecha',0]]

In [27]:
datos_tabla.columns=['Comunidad','Fecha','Fallecidos']

In [28]:
datos_tabla.to_csv('datos_sanidad_tabla.csv')

# Zona de actualización

Utilizaremo este código en el caso de que ya tengamos un fichero histórico

In [9]:
datos=pd.read_csv('datos_sanidad_matriz.csv',index_col=0)

In [10]:
pattern=re.compile(r'(\n\nTotal) ((\n{1,2}\d*[,.]?\d+¥? ){19})')

In [13]:
#descargamos los ficheros que nos faltan
for i in range(263,264):
    descarga(URL_reg.format(i),i)

Descargando: https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Actualizacion_263_COVID-19.pdf
Error con el  263


Actualizamos los datos con los días que nos faltan

In [14]:
for i in range(263,264):
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    page1 = pdf_to_text(fn, pagenum=4)  
    with open(fn1, 'w') as fp:    
        fp.write(page1)
    with open(fn1) as fp:
        text = fp.read()
    cadena=re.search(pattern,text).group(2)         
    print ('Documento {}-->'.format(i),cadena.replace('¥','').replace('\n','').split())
    

NameError: name 'pdf_to_text' is not defined

# Zona de prueba para los días que den error

In [44]:
texto1="Fallecidos  Curados  Nuevos"
texto2="\n\n"
texto3='\n\nESPAÑA \n\nTotal \n\n'
fn='datos_sanidad/'+str(262)+'.pdf'
fn1 = fn.replace('.pdf', '.txt')
page1 = pdf_to_text(fn, pagenum=4)  #con que pagina queremos trabajar?
    #with open(fn1, 'w', encoding='utf-8') as fp:
with open(fn1, 'w') as fp:    
    fp.write(page1)
with open(fn1) as fp:
    text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
lista=text.partition(texto3)[2].replace('\n','').split(' ')
print (1,' -> ',lista[3])


        #print (i,' -> ',lista[0],'-->',lista[1:20])

1  ->  411


In [10]:
pattern=re.compile(r'(\n{1,2}\d*[,.]?\d+¥? ){19}')
pattern=re.compile(r'(\n\nTotal) ((\n{1,2}\d*[,.]?\d+¥? ){19})')

fn='datos_sanidad/'+str(266)+'.pdf'
fn1 = fn.replace('.pdf', '.txt')
text = pdf_to_text(fn, pagenum=3)  #con que pagina queremos trabajar?
cadena=re.search(pattern,text).group(2)         
print ('Documento {}-->'.format(i),cadena.replace('¥','').replace('\n','').split())
datos.loc[:,str(i)]=cadena.replace('¥','').replace('\n','').split()

Documento 83--> ['4.367', '2395', '1.153', '425', '346', '331', '3.839', '4.698', '8.171', '55', '2.460', '982', '1253', '11.445', '42', '635', '908', '2.574', '567']


In [9]:
text

'                                                                                                              \n\n \nSECRETARÍA DE ESTADO \nDE SANIDAD \n \n\nDIRECCIÓN GENERAL DE \nSALUD PÚBLICA  \n\n \n\nCentro  de Coordinación  de   \nAlertas  y Emergencias \nSanitarias \n\nFigura 3. Casos diarios confirmados de COVID-19 por fecha de diagnóstico desde el 11.05.2020 por Comunidades Autónomas, a 06.12.2020 (datos consolidados a las 14:00 horas \ndel 07.12.2020). \n\n \n\n \n\nPágina 5 de 15 \n\n \n\n \n \n\n\x0c'

In [39]:
lista.index('815') #encontramos, dentro de la lista, el indice cuya celda vale 'valor'

ValueError: '815' is not in list

In [ ]:
lista[127:127+19]

In [ ]:
for i in range(50,74):
    texto1="Fallecidos  Curados  Nuevos"
    texto2="\n\n"
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    page1 = pdf_to_text(fn, pagenum=0)
    #with open(fn1, 'w', encoding='utf-8') as fp:
    with open(fn1, 'w') as fp:    
        fp.write(page1)
    with open(fn1) as fp:
        text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
    lista=text.partition(texto1)[2].replace('\n','').split(' ')
    print (i,' -> ',lista[127:127+19])


In [ ]:
text

In [73]:
pattern=re.compile(r'(\n\d*[,.]?\d+¥? ){19}')

In [ ]:
m=pattern.findall(text)

In [ ]:
m

In [ ]:
for match in re.finditer(pattern,text):
    #print ('hola')
    #print (match.start(),match.end())
    #print (text[match.start():match.end()])
    print (text[match.start():match.end()].replace('\n','').split())

In [ ]:
for i in range(46,76):
    texto1="Fallecidos  Curados  Nuevos"
    texto2="\n\n"
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    page1 = pdf_to_text(fn, pagenum=0)
    #with open(fn1, 'w', encoding='utf-8') as fp:
    with open(fn1, 'w') as fp:    
        fp.write(page1)
    with open(fn1) as fp:
        text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
    lista=text.partition(texto1)[2].replace('\n','').split(' ')
    #print (i,' -> ',lista[127:127+19])
    valor=[] #para guardar el primer valor de cada serie, ver cual es mayor luego el que se puede elegir..
    for j,match in enumerate(re.finditer(pattern,text)):
    #print ('hola')
    #print (match.start(),match.end())
        valor[j]=eval(match[0].replace(".","")
    indice=valor.index()
    print ('Documento {}-->'.format(i),text[match.start():match.end()].replace('¥','').replace('\n','').split())

In [161]:
pattern=re.compile(r'(\n{1,2}\d*[,.]?\d+¥? ){19}')
pattern=re.compile(r'(\n\nTotal) ((\n{1,2}\d*[,.]?\d+¥? ){19})')
for i in range(235,263):
    fn='datos_sanidad/'+str(i)+'.pdf'
    fn1 = fn.replace('.pdf', '.txt')
    page1 = pdf_to_text(fn, pagenum=4)  #con que pagina queremos trabajar?
    #with open(fn1, 'w', encoding='utf-8') as fp:
    with open(fn1, 'w') as fp:    
        fp.write(page1)
    with open(fn1) as fp:
        text = fp.read()
    valor={}
    cadena={}
    #for j,match in enumerate(re.finditer(pattern,text)):
    #    cadena=match.group(2)
    cadena=re.search(pattern,text).group(2)         
    print ('Documento {}-->'.format(i),cadena.replace('¥','').replace('\n','').split())
    

Documento 235--> ['2.270', '1566', '391', '352', '267', '248', '3.338', '3.446', '5.960', '13', '1.719', '676', '859', '10.211', '11', '283', '668', '2.032', '442']
Documento 236--> ['2.314', '1598', '409', '356', '270', '248', '3.349', '3.502', '5.961', '16', '1.727', '687', '882', '10.247', '11', '288', '675', '2.038', '453']
Documento 237--> ['2.370', '1613', '414', '357', '273', '250', '3.367', '3.528', '5.968', '16', '1.737', '697', '884', '10.327', '11', '299', '687', '2.046', '454']
Documento 238--> ['2.402', '1620', '419', '357', '277', '251', '3.383', '3.549', '5.972', '16', '1.751', '701', '895', '10.350', '12', '300', '694', '2.061', '456']
Documento 239--> ['2.432', '1627', '427', '357', '278', '252', '3.405', '3.567', '5.977', '18', '1.762', '703', '895', '10.403', '12', '306', '699', '2.061', '458']
Documento 240--> ['2.470', '1654', '437', '362', '282', '252', '3.425', '3.599', '5.991', '19', '1.773', '714', '904', '10.419', '12', '317', '703', '2.082', '463']
Documento 

In [103]:
int('54')

54

In [83]:
dic={1:10,2:20}

In [84]:
max(dic,key=dic.get)

2

In [ ]:
match=pattern.findall(text)

In [ ]:
match

In [ ]:
match

In [ ]:
for i in pattern.finditer(text):
    print (i.group())
    #print (text[i.start():i.end()])

In [150]:
pattern=re.compile(r'(\n{1,2}\d*[,.]?\d+¥? ){19}')
pattern=re.compile(r'(\n\nTotal){1} ((\n{1,2}\d*[,.]?\d+¥? ){19})')
texto1="Fallecidos  Curados  Nuevos"
texto2="\n\n"
texto3='\n\nESPAÑA \n\nTotal \n\n'
fn='datos_sanidad/'+str(239)+'.pdf'
fn1 = fn.replace('.pdf', '.txt')
page1 = pdf_to_text(fn, pagenum=4)  #con que pagina queremos trabajar?
    #with open(fn1, 'w', encoding='utf-8') as fp:
with open(fn1, 'w') as fp:    
    fp.write(page1)
with open(fn1) as fp:
    text = fp.read()
    #lista=text.partition(texto1)[2].partition(texto1)[2].partition(texto1)[2].replace('\n','').split(' ')
lista=text.partition(texto3)[2].replace('\n','').split(' ')
#print (1,' -> ',lista[3])
for match in re.finditer(pattern,text):
    #print ('hola')
    #print (match.start(),match.end())
        print (match.group(2))
        print ('Documento {}[-->'.format(262),text[match.start():match.end()].replace('¥','').replace('\n','').split())

        #print (i,' -> ',lista[0],'-->',lista[1:20])



2.432 

1627 
427 
357 

278 
252 
3.405 

3.567 
5.977 
18 

1.762 
703 
895 

10.403 
12 
306 

699 
2.061 
458 
Documento 262[--> ['Total', '2.432', '1627', '427', '357', '278', '252', '3.405', '3.567', '5.977', '18', '1.762', '703', '895', '10.403', '12', '306', '699', '2.061', '458']


In [112]:
text

'                                                                                                              \n\n \nSECRETARÍA DE ESTADO \nDE SANIDAD \n \nDIRECCIÓN GENERAL DE \nSALUD PÚBLICA  \n\n \n\nCentro de Coordinación de   \nAlertas y Emergencias \nSanitarias \n\nFigura 2. Número de fallecidos diarios por COVID-19 por fecha de defunción en España a 29.11.2020 (datos \nconsolidados a las 14:00 horas del 30.11.2020). \n\n \n\n* En nueve fallecidos diagnosticados en los últimos 7 días no consta la fecha de defunción en la ficha de \nnotificación \n\nTabla 5. Casos de COVID-19 que han fallecido (total y con fecha de \nfallecimiento en los últimos 7 días) por Comunidades Autónomas \nen España a 29.11.2020 (datos consolidados a las 14:00 horas del \n30.11.2020).\n\nCCAA \n\nAndalucía \n\nAragón \nAsturias \nBaleares \n\nCanarias \nCantabria \nCastilla La Mancha \n\nCastilla y León \nCataluña \nCeuta \n\nC. Valenciana \nExtremadura \nGalicia \n\nMadrid \nMelilla \nMurcia \n\nNavarra 